In [1]:
import csv
import math

Hàm load dataset<br>
Tham số:<br>
 + filename: tên file chứa dataset
 + m: số lượng mẫu trong tập huấn luyện
 
Giả sử: training set và test set đều được chứa trong filename, mỗi dòng là mỗi điểm dữ liệu.<br>
Giá trị đầu tiên của mỗi dòng là lớp của điểm dữ liệu đó.<br>
m dòng đầu tiên là training set<br>
Từ dòng thứ m trở đi là test set<br>
Kết quả trả về:<br>
Nếu điểm dữ liệu huấn luận < số điểm dữ liệu trong file => trả về None<br>
Nếu việc đọc thành công sẽ trả về tuple gồm:
+ X_train: dữ liệu điểm huấn luyện
+ Y_train: nhãn của điểm huấn luyện
+ X_test: dữ liệu điểm kiểm thử
+ Y_test: nhãn điểm kiểm thử

In [2]:
def loadDataset(filename, m):
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        if (len(dataset) - m <= 0):
            return None, None, None, None
        for i in range(m):
            X_train.append(dataset[i][1:])
            Y_train.append([dataset[i][0]])
        for i in range(len(dataset) - m):
            X_test.append(dataset[i + m][1:])
            Y_test.append([dataset[i + m][0]])

    return (X_train, Y_train, X_test, Y_test)

Hàm tính khoảng cách 2 điểm dữ liệu<br>
Tham số:
+ instance1: điểm dữ liệu 1
+ instance2: điểm dữ liệu 2
+ n: số thuộc tính (số chiều dữ liệu)
    
Kết quả trả về:<br>
khoảng cách Euclide của hai điểm dữ liệu

In [3]:
def euclideanDistance(instance1, instance2, n):
    distance = 0
    for x in range(n):
        distance += pow((int(instance1[x]) - int(instance2[x])), 2)
    return math.sqrt(distance)

Hàm lấy k láng giềng của điểm dữ liệu test
Tham số:
+ X_train: tập training
+ Y_train: nhãn của tập training
+ test_instance: điểm dữ liệu muốn gắn nhãn đang xét
+ k: số láng giềng
+ m: số điểm dữ liệu training
+ n: số chiều dữ liệu

Kết quả trả về:
một list với mỗi giá trị của nó là một tuple, mỗi tuple bao gồm 
- điểm dữ liệu
- nhãn của điểm dữ liệu
- khoảng cách từ điểm test đang xét tới điểm dữ liệu

In [4]:
def getNeighbors(X_train, Y_train, test_instance, k, m, n):
    distances = []
    
    for x in range(m):
        dist = euclideanDistance(test_instance, X_train[x], n)
        distances.append((X_train[x], Y_train[x], dist))
    distances.sort(key=lambda x: x[2]) #sắp xếp theo khoảng cách tăng dần, tức theo phần thử thứ 2, tính từ 0
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x])
    return neighbors

Hàm lấy nhãn của dữ liệu<br>
Tham số: neighbors: một list, mỗi giá trị của nó là một tuple, mỗi tuple bao gồm 
- điểm dữ liệu
- nhãn của điểm dữ liệu
- khoảng cách từ điểm test đang xét tới điểm dữ liệu

Kết quả trả về: Nhãn của dữ liệu

In [5]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        #neighbors là một list, tại một phần tử bất kì 
        #thì nhãn của dữ liệu (được lưu dưới dạng vector cột, theo numpy), nằm ở vị trí thứ 2 (đếm từ 1)
        response = neighbors[x][1][0]  

        #thực hiện cho các điểm dữ liệu bỏ phiếu quyết định xem điểm test thuộc về lớp nào
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1

    #sắp xếp theo khoảng cách, tức theo phần thử thứ 1 (đếm từ 1), reverse = True tức là sắp xếp giảm dần
    sortedVotes = sorted(classVotes, key=lambda x: x[0], reverse=True) 
    return sortedVotes[0][0] #trả về nhãn nhiều điểm bỏ phiếu nhất

Hàm tính độ chính xác việc dự đoán nhãn<br>
Tham số:
+ Y_test: nhãn thật sự của các điểm test, được lưu dưới dạng vector cột, theo numpy
+ Y_pred: nhãn dự đoán của các điểm test, được lưu dưới dạng vector cột, theo numpy

Kết quả trả về: Tỉ lệ số các điểm dự đoán đúng / tổng số điểm dự đoán

In [16]:
def getAccuracy(Y_test, Y_pred):
    correct = 0
    for x in range(len(Y_test)):
        if Y_test[x][0] == Y_pred[x][0]:
            correct += 1
    return (correct/float(len(Y_test))) * 100.0

In [17]:
filename = 'some.csv'
m = 8124
n = 8
k = 7
X_train, Y_train, X_test, Y_test = loadDataset(filename, m)
if X_train is not None:
    print ('Train set: ' + repr(len(X_train)))
    print ('Test set: ' + repr(len(X_test)))

#     print(Y_test.shape)
    Y_pred = []
    for x in range(len(X_test)):
        neighbors = getNeighbors(X_train, Y_train, X_test[x], k, m, n)
        result = getResponse(neighbors)
        Y_pred.append([result])
        print('> predicted=' + repr(result) + ', actual=' + repr(Y_test[x][0]))
    accuracy = getAccuracy(Y_test, Y_pred)
    print('Accuracy: ' + repr(accuracy) + '%')
#     print(Y_pred)
#     print(Y_test)

Train set: 8124
Test set: 81
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='1', actual='1'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', actual='0'
> predicted='0', ac